In [485]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,mutual_info_regression,f_regression

In [467]:
df=pd.read_csv('ol_retail_sales_dataset.csv')

In [468]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,PaymentMethod,IsDiscounted,TotalAmount
0,189674,70327,Quality-focused actuating approach,8,2023-06-18 14:48:31.780706,89.79,25285,Nicaragua,PayPal,True,646.49
1,64145,86786,Reactive holistic encryption,4,2023-11-25 20:59:42.285805,53.26,2106,Reunion,Cash,False,213.04
2,234612,43632,Horizontal grid-enabled focus group,1,2023-09-15 06:28:34.129252,20.26,14315,Guadeloupe,PayPal,False,20.26
3,652809,32206,Optional uniform application,2,2023-07-01 15:37:28.911531,76.66,46339,Congo,Cash,False,153.32
4,344654,99383,Multi-lateral encompassing architecture,1,2023-09-12 03:48:53.060376,4.96,78064,Vanuatu,Cash,True,4.46


In [469]:
df.shape

(10000, 11)

In [470]:
df.isnull().sum()

InvoiceNo        0
StockCode        0
Description      0
Quantity         0
InvoiceDate      0
UnitPrice        0
CustomerID       0
Country          0
PaymentMethod    0
IsDiscounted     0
TotalAmount      0
dtype: int64

In [471]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   InvoiceNo      10000 non-null  int64  
 1   StockCode      10000 non-null  int64  
 2   Description    10000 non-null  object 
 3   Quantity       10000 non-null  int64  
 4   InvoiceDate    10000 non-null  object 
 5   UnitPrice      10000 non-null  float64
 6   CustomerID     10000 non-null  int64  
 7   Country        10000 non-null  object 
 8   PaymentMethod  10000 non-null  object 
 9   IsDiscounted   10000 non-null  bool   
 10  TotalAmount    10000 non-null  float64
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 791.1+ KB


In [472]:
InvoiceDate=pd.to_datetime(df['InvoiceDate'])

In [473]:
InvoiceDate


0      2023-06-18 14:48:31.780706
1      2023-11-25 20:59:42.285805
2      2023-09-15 06:28:34.129252
3      2023-07-01 15:37:28.911531
4      2023-09-12 03:48:53.060376
                  ...            
9995   2024-04-03 10:51:06.159388
9996   2024-04-01 00:44:46.790282
9997   2023-08-16 01:17:21.676894
9998   2023-11-01 13:06:27.265889
9999   2023-10-05 08:26:33.666383
Name: InvoiceDate, Length: 10000, dtype: datetime64[ns]

In [474]:
df['IsDiscounted'] = df['IsDiscounted'].map({True: 1, False: 0})
La=LabelEncoder()
df['Country']=La.fit_transform(df['Country'])


In [475]:

encoder = OneHotEncoder(sparse=False)
encoded_v = encoder.fit_transform(df[['PaymentMethod']])
encoded_v = pd.DataFrame(encoded_v, columns=encoder.get_feature_names_out(['PaymentMethod']))
df = pd.concat([df, encoded_v], axis=1)
df.drop('PaymentMethod', axis=1, inplace=True)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [476]:
from sklearn.preprocessing import StandardScaler
mm= StandardScaler()
df[['Quantity','UnitPrice','TotalAmount']] = mm.fit_transform(df[['Quantity','UnitPrice','TotalAmount']])
df


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsDiscounted,TotalAmount,PaymentMethod_Cash,PaymentMethod_Mastercard,PaymentMethod_PayPal,PaymentMethod_Visa
0,189674,70327,Quality-focused actuating approach,0.870435,2023-06-18 14:48:31.780706,1.368024,25285,155,1,1.763338,0.0,0.0,1.0,0.0
1,64145,86786,Reactive holistic encryption,-0.523488,2023-11-25 20:59:42.285805,0.090548,2106,177,0,-0.232357,1.0,0.0,0.0,0.0
2,234612,43632,Horizontal grid-enabled focus group,-1.568930,2023-09-15 06:28:34.129252,-1.063481,14315,85,0,-1.119957,0.0,0.0,1.0,0.0
3,652809,32206,Optional uniform application,-1.220450,2023-07-01 15:37:28.911531,0.908860,46339,49,0,-0.507320,1.0,0.0,0.0,0.0
4,344654,99383,Multi-lateral encompassing architecture,-1.568930,2023-09-12 03:48:53.060376,-1.598531,78064,235,1,-1.192703,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,189309,33617,Synergistic client-driven support,-0.175007,2024-04-03 10:51:06.159388,1.204362,50473,36,1,0.550175,1.0,0.0,0.0,0.0
9996,147053,45864,Progressive next generation adapter,0.173474,2024-04-01 00:44:46.790282,-1.468441,94857,69,0,-0.973451,0.0,0.0,0.0,1.0
9997,192462,34790,Visionary bottom-line application,0.521955,2023-08-16 01:17:21.676894,-1.107194,19869,203,0,-0.600556,0.0,1.0,0.0,0.0
9998,630982,99631,Ameliorated interactive workforce,-0.871969,2023-11-01 13:06:27.265889,-1.520197,68996,93,1,-1.123732,0.0,0.0,1.0,0.0


In [477]:

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsDiscounted,TotalAmount,PaymentMethod_Cash,PaymentMethod_Mastercard,PaymentMethod_PayPal,PaymentMethod_Visa
0,189674,70327,Quality-focused actuating approach,0.870435,2023-06-18 14:48:31.780706,1.368024,25285,155,1,1.763338,0.0,0.0,1.0,0.0
1,64145,86786,Reactive holistic encryption,-0.523488,2023-11-25 20:59:42.285805,0.090548,2106,177,0,-0.232357,1.0,0.0,0.0,0.0
2,234612,43632,Horizontal grid-enabled focus group,-1.568930,2023-09-15 06:28:34.129252,-1.063481,14315,85,0,-1.119957,0.0,0.0,1.0,0.0
3,652809,32206,Optional uniform application,-1.220450,2023-07-01 15:37:28.911531,0.908860,46339,49,0,-0.507320,1.0,0.0,0.0,0.0
4,344654,99383,Multi-lateral encompassing architecture,-1.568930,2023-09-12 03:48:53.060376,-1.598531,78064,235,1,-1.192703,1.0,0.0,0.0,0.0


In [478]:
x=df.drop(['Description','InvoiceDate','TotalAmount'],axis=1)
y=df['TotalAmount']

In [487]:

k = 5
selector = SelectKBest(score_func=mutual_info_regression, k=k)
X_s = selector.fit_transform(x, y)
selected_features =x.columns[selector.get_support(indices=True)].tolist()
# Print the selected features
print("Selected features:", selected_features)

Selected features: ['Quantity', 'UnitPrice', 'Country', 'IsDiscounted', 'PaymentMethod_PayPal']
